In [16]:
# -*- coding: utf-8 -*-
"""
Análise e estratificação pré-RNA (SEM GRÁFICOS)
Base: Dataset_Vetor.json
Foco: erros em metros de latitude, longitude e erro radial
NÃO altera nomes/colunas do dataset.
"""

import json
from pathlib import Path
import math
import numpy as np
import pandas as pd

# =========================
# Configuração e carregamento
# =========================
JSON_PATH = "Dataset_Vetor.json"  # ajuste se necessário
OUTPUT_DIR = Path("./parte1_resultados_sem_graficos")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("Carregando dataset...")
with open(JSON_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)
df = pd.DataFrame(data)
print(f"✅ Dataset carregado: {len(df):,} registros, {df.shape[1]} colunas")

# =========================
# Colunas de erro (NÃO renomear)
# =========================
lat_col = "diferencalatitudeMetros"
lon_col = "diferencalongitudeMetros"

missing_cols = [c for c in [lat_col, lon_col] if c not in df.columns]
if missing_cols:
    raise KeyError(f"As colunas esperadas não existem no dataset: {missing_cols}")

# Garantir tipos numéricos e remover NaNs/inf
for c in [lat_col, lon_col]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=[lat_col, lon_col]).copy()
print(f"✅ Após limpeza (NaN/inf): {len(df):,} registros")

# =========================
# Funções utilitárias
# =========================
def tukey_limits(series: pd.Series):
    q1, q3 = np.percentile(series, [25, 75])
    iqr = q3 - q1
    return q1 - 1.5 * iqr, q3 + 1.5 * iqr

def summarize_errors(y: pd.Series, name: str) -> pd.DataFrame:
    """Resumo estatístico + métricas de erro quadráticas (baseline 0)."""
    y = pd.to_numeric(y, errors="coerce").dropna()
    n = y.size
    if n == 0:
        raise ValueError(f"Sem dados válidos para {name}.")

    # Estatísticas básicas
    mean = y.mean()
    median = y.median()
    std = y.std(ddof=1)
    var = y.var(ddof=1)
    mn = y.min()
    mx = y.max()
    q = y.quantile([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99])
    iqr = q.loc[0.75] - q.loc[0.25]
    mad = (y - mean).abs().mean()  # mean absolute deviation em relação à média

    # Métricas de erro (baseline 0)
    abs_y = y.abs()
    mae = abs_y.mean()
    mse = (y ** 2).mean()
    rmse = math.sqrt(mse)

    # R² do baseline 0 (pode ser negativo; útil para referência)
    # R² = 1 - SSE/SST, com SSE = sum((y-0)^2) e SST = sum((y-mean)^2)
    sse = (y ** 2).sum()
    sst = ((y - mean) ** 2).sum()
    r2_baseline0 = float("nan") if sst == 0 else 1 - (sse / sst)

    # Outliers (Tukey)
    low, high = tukey_limits(y)
    outliers = ((y < low) | (y > high)).sum()

    # Faixas de erro (limiares em metros)
    thresholds = [0.5, 1, 2, 3, 5, 10, 20, 50, 100]
    within = {f"perc_|e|<={t}m": (abs_y.le(t).mean() * 100.0) for t in thresholds}

    # Bins por magnitude do erro absoluto
    # Ajuste as bordas se desejar granularidade diferente
    bins = [-np.inf, 0.5, 1, 2, 3, 5, 10, 20, 50, 100, np.inf]
    labels = ["<=0.5", "0.5–1", "1–2", "2–3", "3–5", "5–10", "10–20", "20–50", "50–100", ">100"]
    dist_bins = pd.cut(abs_y, bins=bins, labels=labels, include_lowest=True)
    bin_counts = dist_bins.value_counts().reindex(labels).fillna(0).astype(int)
    bin_perc = (bin_counts / n * 100).round(4)

    # Monta DataFrame de métricas
    summary = {
        "n": n,
        "mean(m)": mean,
        "median(m)": median,
        "std(m)": std,
        "var(m^2)": var,
        "min(m)": mn,
        "p1(m)": q.loc[0.01],
        "p5(m)": q.loc[0.05],
        "p10(m)": q.loc[0.10],
        "p25(m)": q.loc[0.25],
        "p50(m)": q.loc[0.50],
        "p75(m)": q.loc[0.75],
        "p90(m)": q.loc[0.90],
        "p95(m)": q.loc[0.95],
        "p99(m)": q.loc[0.99],
        "max(m)": mx,
        "iqr(m)": iqr,
        "mad(m)": mad,
        "MAE(m)": mae,
        "MSE(m^2)": mse,
        "RMSE(m)": rmse,
        "R2(baseline_0)": r2_baseline0,
        "outliers_tukey(n)": int(outliers),
        "outliers_tukey(%)": round(outliers / n * 100.0, 4),
    }
    summary.update({k: round(v, 4) for k, v in within.items()})

    # Retorna duas tabelas: (1) métricas gerais, (2) distribuição por bins
    summary_df = pd.DataFrame(summary, index=[name])
    bins_df = pd.DataFrame({
        "bin_abs_error(m)": labels,
        "count": bin_counts.values,
        "perc(%)": bin_perc.values
    })
    return summary_df, bins_df

def quadrant_breakdown(lat: pd.Series, lon: pd.Series) -> pd.DataFrame:
    """Percentual por quadrante do plano (sinal de lat/lon)."""
    q1 = ((lat >= 0) & (lon >= 0)).mean() * 100
    q2 = ((lat >= 0) & (lon < 0)).mean() * 100
    q3 = ((lat < 0) & (lon < 0)).mean() * 100
    q4 = ((lat < 0) & (lon >= 0)).mean() * 100
    return pd.DataFrame({
        "quadrante": ["Q1(+lat,+lon)", "Q2(+lat,-lon)", "Q3(-lat,-lon)", "Q4(-lat,+lon)"],
        "perc(%)": [round(q1, 4), round(q2, 4), round(q3, 4), round(q4, 4)]
    })

def correlation_summary(lat: pd.Series, lon: pd.Series) -> pd.DataFrame:
    """Correlação entre erros lat/lon e variâncias."""
    lat = pd.to_numeric(lat, errors="coerce")
    lon = pd.to_numeric(lon, errors="coerce")
    mask = lat.notna() & lon.notna()
    lat = lat[mask]
    lon = lon[mask]
    pearson = float(pd.Series(lat).corr(pd.Series(lon), method="pearson"))
    spearman = float(pd.Series(lat).corr(pd.Series(lon), method="spearman"))
    return pd.DataFrame({
        "metric": ["pearson_r", "spearman_rho", "var_lat", "var_lon", "cov_lat_lon"],
        "value": [
            round(pearson, 6),
            round(spearman, 6),
            round(lat.var(ddof=1), 6),
            round(lon.var(ddof=1), 6),
            round(np.cov(lat, lon, ddof=1)[0, 1], 6),
        ]
    })

# =========================
# 1) ANÁLISE LATITUDE (erro em metros)
# =========================
lat_summary, lat_bins = summarize_errors(df[lat_col], name="latitude_erro(m)")
lat_summary.to_csv(OUTPUT_DIR / "stats_latitude.csv", index=True)
lat_bins.to_csv(OUTPUT_DIR / "bins_latitude.csv", index=False)

print("\n=== RESUMO LATITUDE (m) ===")
print(lat_summary.to_string())
print("\n--- Distribuição por faixas (|erro| em m) LATITUDE ---")
print(lat_bins.to_string(index=False))

# =========================
# 2) ANÁLISE LONGITUDE (erro em metros)
# =========================
lon_summary, lon_bins = summarize_errors(df[lon_col], name="longitude_erro(m)")
lon_summary.to_csv(OUTPUT_DIR / "stats_longitude.csv", index=True)
lon_bins.to_csv(OUTPUT_DIR / "bins_longitude.csv", index=False)

print("\n=== RESUMO LONGITUDE (m) ===")
print(lon_summary.to_string())
print("\n--- Distribuição por faixas (|erro| em m) LONGITUDE ---")
print(lon_bins.to_string(index=False))

# =========================
# 3) ERRO RADIAL (m) + RELAÇÃO LAT × LON
# =========================
# Erro radial em metros (hipotenusa): sqrt(lat^2 + lon^2)
df["erro_radial_m"] = np.sqrt(df[lat_col]**2 + df[lon_col]**2)

rad_summary, rad_bins = summarize_errors(df["erro_radial_m"], name="erro_radial(m)")
rad_summary.to_csv(OUTPUT_DIR / "stats_erro_radial.csv", index=True)
rad_bins.to_csv(OUTPUT_DIR / "bins_erro_radial.csv", index=False)

print("\n=== RESUMO ERRO RADIAL (m) ===")
print(rad_summary.to_string())
print("\n--- Distribuição por faixas (erro radial em m) ---")
print(rad_bins.to_string(index=False))

# Correlação e quadrantes
corr_df = correlation_summary(df[lat_col], df[lon_col])
corr_df.to_csv(OUTPUT_DIR / "correlacao_lat_lon.csv", index=False)

quads_df = quadrant_breakdown(df[lat_col], df[lon_col])
quads_df.to_csv(OUTPUT_DIR / "quadrantes_lat_lon.csv", index=False)

print("\n=== CORRELAÇÃO LAT × LON ===")
print(corr_df.to_string(index=False))
print("\n=== QUADRANTES (sinais de lat/lon) ===")
print(quads_df.to_string(index=False))

# =========================
# 4) (Opcional) Estratificação temporal simples (se existir 'timestamp' normalizado)
#     – Sem gráficos; apenas métricas por faixas de tempo
# =========================
if "timestamp" in df.columns:
    # Exemplo de bucketing: quintis (ou defina manualmente buckets de hora)
    df["_bucket_tempo"] = pd.qcut(df["timestamp"], q=5, labels=[f"Q{i}" for i in range(1, 6)])
    group_cols = ["_bucket_tempo"]

    def group_metrics(series: pd.Series) -> pd.Series:
        series = pd.to_numeric(series, errors="coerce").dropna()
        return pd.Series({
            "n": series.size,
            "mean": series.mean(),
            "median": series.median(),
            "std": series.std(ddof=1),
            "MAE": series.abs().mean(),
            "MSE": (series**2).mean(),
            "RMSE": math.sqrt((series**2).mean()),
        })

    lat_by_time = df.groupby(group_cols)[lat_col].apply(group_metrics).unstack()
    lon_by_time = df.groupby(group_cols)[lon_col].apply(group_metrics).unstack()
    rad_by_time = df.groupby(group_cols)["erro_radial_m"].apply(group_metrics).unstack()

    lat_by_time.to_csv(OUTPUT_DIR / "lat_por_tempo.csv")
    lon_by_time.to_csv(OUTPUT_DIR / "lon_por_tempo.csv")
    rad_by_time.to_csv(OUTPUT_DIR / "radial_por_tempo.csv")

    print("\n=== ESTRATIFICAÇÃO TEMPORAL (timestamp em quintis) ===")
    print("\n— LAT por tempo —")
    print(lat_by_time.to_string())
    print("\n— LON por tempo —")
    print(lon_by_time.to_string())
    print("\n— RADIAL por tempo —")
    print(rad_by_time.to_string())
else:
    print("\n⚠️ Coluna 'timestamp' não encontrada. Pulo da estratificação temporal.")

# =========================
# 5) Export básico do describe numérico (apoio)
# =========================
num_desc = df.select_dtypes(include=[np.number]).describe().T
num_desc.to_csv(OUTPUT_DIR / "describe_numeric.csv", index=True)
print("\n💾 Arquivos salvos em:", OUTPUT_DIR.resolve())


Carregando dataset...
✅ Dataset carregado: 419,878 registros, 647 colunas
✅ Após limpeza (NaN/inf): 419,878 registros

=== RESUMO LATITUDE (m) ===
                       n  mean(m)  median(m)    std(m)   var(m^2)  min(m)  p1(m)  p5(m)  p10(m)  p25(m)  p50(m)  p75(m)  p90(m)  p95(m)  p99(m)  max(m)  iqr(m)    mad(m)   MAE(m)   MSE(m^2)   RMSE(m)  R2(baseline_0)  outliers_tukey(n)  outliers_tukey(%)  perc_|e|<=0.5m  perc_|e|<=1m  perc_|e|<=2m  perc_|e|<=3m  perc_|e|<=5m  perc_|e|<=10m  perc_|e|<=20m  perc_|e|<=50m  perc_|e|<=100m
latitude_erro(m)  419878   2.5426       2.11  5.414513  29.316953  -87.88  -5.69  -2.46   -1.58    0.51    2.11     4.7    6.58    7.57    19.8    78.2    4.19  2.907784  3.55032  35.781699  5.981781       -0.220515               9424             2.2445         10.0572       18.0324       40.8157       57.5015        75.681        98.1028        98.6891        99.8128           100.0

--- Distribuição por faixas (|erro| em m) LATITUDE ---
bin_abs_error(m)  count

In [17]:
# -*- coding: utf-8 -*-
"""
Análise e estratificação pré-RNA por período (MANHÃ vs TARDE)
Corte temporal: timestamp < 0.5 (manhã) / >= 0.5 (tarde)
Foco: métricas de erro em metros (sem gráficos)
"""

import json
from pathlib import Path
import math
import numpy as np
import pandas as pd

# =========================
# Configuração e carregamento
# =========================
JSON_PATH = "Dataset_Vetor.json"  # ajuste se necessário
BASE_OUTPUT = Path("./parte1_resultados_periodos")
BASE_OUTPUT.mkdir(parents=True, exist_ok=True)

print("Carregando dataset...")
with open(JSON_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)
df = pd.DataFrame(data)
print(f"✅ Dataset carregado: {len(df):,} registros, {df.shape[1]} colunas")

lat_col = "diferencalatitudeMetros"
lon_col = "diferencalongitudeMetros"

missing_cols = [c for c in [lat_col, lon_col, "timestamp"] if c not in df.columns]
if missing_cols:
    raise KeyError(f"As colunas esperadas não existem no dataset: {missing_cols}")

# Limpeza básica
for c in [lat_col, lon_col, "timestamp"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")
df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=[lat_col, lon_col, "timestamp"])
print(f"✅ Após limpeza: {len(df):,} registros válidos")

# =========================
# Funções utilitárias
# =========================
def tukey_limits(series):
    q1, q3 = np.percentile(series, [25, 75])
    iqr = q3 - q1
    return q1 - 1.5 * iqr, q3 + 1.5 * iqr

def summarize_errors(y: pd.Series, name: str) -> pd.DataFrame:
    y = pd.to_numeric(y, errors="coerce").dropna()
    n = len(y)
    if n == 0:
        raise ValueError(f"Sem dados válidos para {name}")

    mean = y.mean()
    median = y.median()
    std = y.std(ddof=1)
    var = y.var(ddof=1)
    mn = y.min()
    mx = y.max()
    q = y.quantile([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99])
    iqr = q.loc[0.75] - q.loc[0.25]
    mad = (y - mean).abs().mean()

    abs_y = y.abs()
    mae = abs_y.mean()
    mse = (y ** 2).mean()
    rmse = math.sqrt(mse)

    sse = (y ** 2).sum()
    sst = ((y - mean) ** 2).sum()
    r2_baseline0 = float("nan") if sst == 0 else 1 - (sse / sst)

    low, high = tukey_limits(y)
    outliers = ((y < low) | (y > high)).sum()

    thresholds = [0.5, 1, 2, 3, 5, 10, 20, 50, 100]
    within = {f"perc_|e|<={t}m": abs_y.le(t).mean() * 100.0 for t in thresholds}

    bins = [-np.inf, 0.5, 1, 2, 3, 5, 10, 20, 50, 100, np.inf]
    labels = ["<=0.5", "0.5–1", "1–2", "2–3", "3–5", "5–10", "10–20", "20–50", "50–100", ">100"]
    dist_bins = pd.cut(abs_y, bins=bins, labels=labels, include_lowest=True)
    bin_counts = dist_bins.value_counts().reindex(labels).fillna(0).astype(int)
    bin_perc = (bin_counts / n * 100).round(4)

    summary = {
        "n": n,
        "mean(m)": mean,
        "median(m)": median,
        "std(m)": std,
        "var(m^2)": var,
        "min(m)": mn,
        "p25(m)": q.loc[0.25],
        "p50(m)": q.loc[0.50],
        "p75(m)": q.loc[0.75],
        "max(m)": mx,
        "iqr(m)": iqr,
        "mad(m)": mad,
        "MAE(m)": mae,
        "MSE(m^2)": mse,
        "RMSE(m)": rmse,
        "R2(baseline_0)": r2_baseline0,
        "outliers_tukey(n)": int(outliers),
        "outliers_tukey(%)": round(outliers / n * 100.0, 4),
    }
    summary.update({k: round(v, 4) for k, v in within.items()})
    summary_df = pd.DataFrame(summary, index=[name])

    bins_df = pd.DataFrame({
        "bin_abs_error(m)": labels,
        "count": bin_counts.values,
        "perc(%)": bin_perc.values
    })
    return summary_df, bins_df

def correlation_summary(lat: pd.Series, lon: pd.Series) -> pd.DataFrame:
    lat = pd.to_numeric(lat, errors="coerce")
    lon = pd.to_numeric(lon, errors="coerce")
    mask = lat.notna() & lon.notna()
    lat, lon = lat[mask], lon[mask]
    pearson = float(pd.Series(lat).corr(pd.Series(lon), method="pearson"))
    spearman = float(pd.Series(lat).corr(pd.Series(lon), method="spearman"))
    return pd.DataFrame({
        "metric": ["pearson_r", "spearman_rho", "var_lat", "var_lon", "cov_lat_lon"],
        "value": [
            round(pearson, 6),
            round(spearman, 6),
            round(lat.var(ddof=1), 6),
            round(lon.var(ddof=1), 6),
            round(np.cov(lat, lon, ddof=1)[0, 1], 6),
        ]
    })

# =========================
# Separar períodos (manhã/tarde)
# =========================
df["periodo"] = np.where(df["timestamp"] < 0.5, "manha", "tarde")
print(df["periodo"].value_counts())

# =========================
# Função de análise por período
# =========================
def analisar_periodo(sub_df: pd.DataFrame, periodo: str):
    out_dir = BASE_OUTPUT / periodo
    out_dir.mkdir(parents=True, exist_ok=True)
    print(f"\n🔹 Analisando período: {periodo.upper()} ({len(sub_df):,} registros)")

    # Latitude
    lat_summary, lat_bins = summarize_errors(sub_df[lat_col], f"latitude_erro({periodo})")
    lat_summary.to_csv(out_dir / "stats_latitude.csv", index=True)
    lat_bins.to_csv(out_dir / "bins_latitude.csv", index=False)

    # Longitude
    lon_summary, lon_bins = summarize_errors(sub_df[lon_col], f"longitude_erro({periodo})")
    lon_summary.to_csv(out_dir / "stats_longitude.csv", index=True)
    lon_bins.to_csv(out_dir / "bins_longitude.csv", index=False)

    # Erro radial
    sub_df["erro_radial_m"] = np.sqrt(sub_df[lat_col]**2 + sub_df[lon_col]**2)
    rad_summary, rad_bins = summarize_errors(sub_df["erro_radial_m"], f"erro_radial({periodo})")
    rad_summary.to_csv(out_dir / "stats_erro_radial.csv", index=True)
    rad_bins.to_csv(out_dir / "bins_erro_radial.csv", index=False)

    # Correlação lat x lon
    corr_df = correlation_summary(sub_df[lat_col], sub_df[lon_col])
    corr_df.to_csv(out_dir / "correlacao_lat_lon.csv", index=False)

    print("\n--- LAT ---")
    print(lat_summary.to_string())
    print("\n--- LON ---")
    print(lon_summary.to_string())
    print("\n--- RADIAL ---")
    print(rad_summary.to_string())
    print("\n--- CORRELAÇÃO LAT × LON ---")
    print(corr_df.to_string(index=False))

    # Salva resumo consolidado
    combined = pd.concat([lat_summary, lon_summary, rad_summary])
    combined.to_csv(out_dir / "resumo_geral.csv")

# =========================
# Execução por período
# =========================
for periodo in ["manha", "tarde"]:
    subset = df[df["periodo"] == periodo]
    if len(subset) > 0:
        analisar_periodo(subset, periodo)
    else:
        print(f"⚠️ Nenhum dado disponível para {periodo}")

print("\n💾 Resultados salvos em:", BASE_OUTPUT.resolve())


Carregando dataset...
✅ Dataset carregado: 419,878 registros, 647 colunas
✅ Após limpeza: 419,878 registros válidos
periodo
manha    214258
tarde    205620
Name: count, dtype: int64

🔹 Analisando período: MANHA (214,258 registros)

--- LAT ---
                           n   mean(m)  median(m)    std(m)   var(m^2)  min(m)  p25(m)  p50(m)  p75(m)  max(m)  iqr(m)    mad(m)    MAE(m)   MSE(m^2)   RMSE(m)  R2(baseline_0)  outliers_tukey(n)  outliers_tukey(%)  perc_|e|<=0.5m  perc_|e|<=1m  perc_|e|<=2m  perc_|e|<=3m  perc_|e|<=5m  perc_|e|<=10m  perc_|e|<=20m  perc_|e|<=50m  perc_|e|<=100m
latitude_erro(manha)  214258  0.559631       0.83  4.099707  16.807597  -87.88   -0.84    0.83    1.57   74.74    2.41  1.783481  1.919722  17.120705  4.137717       -0.018634               5080              2.371         16.6706       28.6197       68.2747       89.8748       97.6729        98.4085        99.2075        99.8964           100.0

--- LON ---
                            n   mean(m)  median(m

C:\Users\bruno\AppData\Local\Temp\ipykernel_30748\926002793.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df["erro_radial_m"] = np.sqrt(sub_df[lat_col]**2 + sub_df[lon_col]**2)



🔹 Analisando período: TARDE (205,620 registros)

--- LAT ---
                           n   mean(m)  median(m)    std(m)   var(m^2)  min(m)  p25(m)  p50(m)  p75(m)  max(m)  iqr(m)    mad(m)    MAE(m)   MSE(m^2)   RMSE(m)  R2(baseline_0)  outliers_tukey(n)  outliers_tukey(%)  perc_|e|<=0.5m  perc_|e|<=1m  perc_|e|<=2m  perc_|e|<=3m  perc_|e|<=5m  perc_|e|<=10m  perc_|e|<=20m  perc_|e|<=50m  perc_|e|<=100m
latitude_erro(tarde)  205620  4.608874        4.7  5.829673  33.985083  -85.46    2.92     4.7    6.09    78.2    3.17  2.536258  5.249418  55.226633  7.431462       -0.625034               6644             3.2312           3.166        7.0003       12.2031       23.7681       52.7653        97.7843         98.149        99.7257           100.0

--- LON ---
                            n   mean(m)  median(m)    std(m)  var(m^2)  min(m)  p25(m)  p50(m)  p75(m)  max(m)  iqr(m)    mad(m)    MAE(m)   MSE(m^2)   RMSE(m)  R2(baseline_0)  outliers_tukey(n)  outliers_tukey(%)  perc_|e|<=0.5m  

C:\Users\bruno\AppData\Local\Temp\ipykernel_30748\926002793.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df["erro_radial_m"] = np.sqrt(sub_df[lat_col]**2 + sub_df[lon_col]**2)


In [18]:
# -*- coding: utf-8 -*-
"""
Comparativo estatístico entre períodos (manhã vs tarde)
Base: Dataset_Vetor.json
Corte: timestamp < 0.5 (manhã) / >= 0.5 (tarde)
Foco: comparações de quartis, dispersão e métricas de erro GNSS
"""

import json
import math
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn-v0_8-whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

# ===============================
# Configurações
# ===============================
JSON_PATH = "Dataset_Vetor.json"
OUTPUT_DIR = Path("./parte1_resultados_comparativo")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

lat_col = "diferencalatitudeMetros"
lon_col = "diferencalongitudeMetros"

# ===============================
# Carregamento e limpeza
# ===============================
print("Carregando dataset...")
with open(JSON_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)
df = pd.DataFrame(data)

required = [lat_col, lon_col, "timestamp"]
missing = [c for c in required if c not in df.columns]
if missing:
    raise KeyError(f"Faltando colunas obrigatórias: {missing}")

for c in required:
    df[c] = pd.to_numeric(df[c], errors="coerce")

df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=required)
print(f"✅ Registros válidos: {len(df):,}")

df["periodo"] = np.where(df["timestamp"] < 0.5, "manhã", "tarde")
df["erro_radial_m"] = np.sqrt(df[lat_col]**2 + df[lon_col]**2)

# ===============================
# Funções auxiliares
# ===============================
def resumo_quartil(series):
    q = series.quantile([0.25, 0.5, 0.75])
    iqr = q.loc[0.75] - q.loc[0.25]
    return {
        "mean": series.mean(),
        "median": q.loc[0.5],
        "std": series.std(ddof=1),
        "var": series.var(ddof=1),
        "Q1": q.loc[0.25],
        "Q2": q.loc[0.5],
        "Q3": q.loc[0.75],
        "IQR": iqr,
        "min": series.min(),
        "max": series.max(),
        "range": series.max() - series.min(),
        "mad": (series - series.mean()).abs().mean(),
    }

def comparar_periodos(df, coluna):
    manha = df[df["periodo"] == "manhã"][coluna]
    tarde = df[df["periodo"] == "tarde"][coluna]

    stats = {
        "metric": [],
        "manhã": [],
        "tarde": [],
        "diferença": [],
        "diferença_%": [],
    }

    m = resumo_quartil(manha)
    t = resumo_quartil(tarde)

    for key in m.keys():
        diff = t[key] - m[key]
        perc = (diff / abs(m[key]) * 100) if m[key] != 0 else np.nan
        stats["metric"].append(key)
        stats["manhã"].append(round(m[key], 5))
        stats["tarde"].append(round(t[key], 5))
        stats["diferença"].append(round(diff, 5))
        stats["diferença_%"].append(round(perc, 2))

    df_comp = pd.DataFrame(stats)
    return df_comp

# ===============================
# Comparação LATITUDE / LONGITUDE / RADIAL
# ===============================
comparativos = {}
for var in [lat_col, lon_col, "erro_radial_m"]:
    comp = comparar_periodos(df, var)
    comparativos[var] = comp
    comp.to_csv(OUTPUT_DIR / f"comparativo_{var}.csv", index=False)
    print(f"\n=== COMPARATIVO: {var} ===")
    print(comp.to_string(index=False))

# ===============================
# Visualizações (para o TCC)
# ===============================
sns.set_palette("muted")

def gerar_boxplots(col, nome):
    plt.figure(figsize=(8, 6))
    sns.boxplot(x="periodo", y=col, data=df, width=0.5)
    plt.title(f"Boxplot do erro em {nome} (m) – manhã vs tarde")
    plt.xlabel("")
    plt.ylabel("Erro (m)")
    plt.savefig(OUTPUT_DIR / f"boxplot_{nome}.png", dpi=300, bbox_inches="tight")
    plt.close()

def gerar_violinplots(col, nome):
    plt.figure(figsize=(8, 6))
    sns.violinplot(x="periodo", y=col, data=df, inner="quartile")
    plt.title(f"Distribuição (Violin) – {nome} GNSS")
    plt.ylabel("Erro (m)")
    plt.savefig(OUTPUT_DIR / f"violin_{nome}.png", dpi=300, bbox_inches="tight")
    plt.close()

def gerar_barras(col, nome):
    agg = df.groupby("periodo")[col].agg(["mean", "std"])
    agg["sem"] = agg["std"] / np.sqrt(df.groupby("periodo")[col].count())
    plt.figure(figsize=(6, 5))
    plt.bar(agg.index, agg["mean"], yerr=agg["sem"], capsize=6, color=["#1f77b4", "#ff7f0e"])
    plt.title(f"Média ± erro padrão – {nome} GNSS")
    plt.ylabel("Erro médio (m)")
    plt.savefig(OUTPUT_DIR / f"barras_{nome}.png", dpi=300, bbox_inches="tight")
    plt.close()

for col, nome in zip([lat_col, lon_col, "erro_radial_m"], ["Latitude", "Longitude", "Erro Radial"]):
    gerar_boxplots(col, nome)
    gerar_violinplots(col, nome)
    gerar_barras(col, nome)

print("\n💾 Arquivos comparativos e figuras salvos em:", OUTPUT_DIR.resolve())


Carregando dataset...
✅ Registros válidos: 419,878

=== COMPARATIVO: diferencalatitudeMetros ===
metric     manhã     tarde  diferença  diferença_%
  mean   0.55963   4.60887    4.04924       723.56
median   0.83000   4.70000    3.87000       466.27
   std   4.09971   5.82967    1.72997        42.20
   var  16.80760  33.98508   17.17749       102.20
    Q1  -0.84000   2.92000    3.76000       447.62
    Q2   0.83000   4.70000    3.87000       466.27
    Q3   1.57000   6.09000    4.52000       287.90
   IQR   2.41000   3.17000    0.76000        31.54
   min -87.88000 -85.46000    2.42000         2.75
   max  74.74000  78.20000    3.46000         4.63
 range 162.62000 163.66000    1.04000         0.64
   mad   1.78348   2.53626    0.75278        42.21

=== COMPARATIVO: diferencalongitudeMetros ===
metric      manhã      tarde  diferença  diferença_%
  mean    0.59107   -0.40736   -0.99843      -168.92
median    0.39000   -0.62000   -1.01000      -258.97
   std    5.19197    7.97224    2.